<a href="https://colab.research.google.com/github/yajima-yasutoshi/DataMining2024/blob/main/20241112/%E3%82%AF%E3%83%A9%E3%82%B9%E3%82%BF%E3%83%AA%E3%83%B3%E3%82%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# データマイニング第7回（20241112）

#本日の講義の目的

クラスタリング手法に関する説明を行う

## 本日の講義の資料

以下のサイトに保存してある「利用回数.xlsx」をダウンロード使う。

https://github.com/yajima-yasutoshi/DataMining/tree/main/20241112



# 準備

In [ ]:
# インストール
!pip install japanize-matplotlib

## 必要なライブラリーのインポート

In [ ]:
# 必要なライブラリのインポート
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import japanize_matplotlib
import seaborn as sns

In [ ]:
# prompt: Pandasの表示形式を小数点以下1桁にする

import pandas as pd
pd.options.display.float_format = '{:.1f}'.format


# クラスタリングとは

## データ準備
まず、以下のサイトより「利用回数.xlsx」を各自のPCダウンロードし、
Google Driveにアップロード、Google Colab に読み込みを行う。

https://github.com/yajima-yasutoshi/DataMining/tree/main/20241112


In [ ]:
# 読み込むファイルを指定する
file_path = '/content/drive/MyDrive/周南公立大学/講義/データマイニング/データ/利用回数.xlsx'
df = pd.read_excel(file_path)

In [ ]:
df.info()

「利用回数.xlsx」には、顧客毎に**年齢**とあるサービスの**利用回数**が記録されている。

In [ ]:
df.head()

## 散布図による可視化

年齢と利用回数の関係を**散布図**で可視化する。

In [ ]:
sns.scatterplot(x='年齢', y='利用回数', data=df)
plt.grid(True)  # グリッド線を追加
plt.title('散布図')
plt.show()

上の散布図から、全体の利用者は
 * 年齢が高く利用回数の多い利用者
 * 年齢が低く利用回数の多い利用者
 * 年齢が低く利用回数の少ない利用者

に傾向が分かれていることが観察できる。

傾向が似通っているレコード(この例では利用者)を似た性質のグループに分けて、
全体をいくつかのまとまりとして理解する手法が
**「クラスタリング」**である。

スーパーやドラッグストア、コンビニなどの小売店などでは、
顧客の属性（性別や年齢など）や
購買行動（利用回数や購買金額など）のデータを元にして、
傾向が似ている顧客をいくつかのグループ（クラスター）に分類し、
どのような顧客が良く利用しているのか、
どのような商品を購入する顧客がいるのか、
など、顧客の行動を理解することで販促や商品開発などを行っている。

参考事例：
https://www.nikkei.com/article/DGXZQOUC132X30T10C22A9000000/

# データを標準化する

 クラスタリングを実行する事前の準備としてデータの**標準化**が必要である。
 標準化とは、データの大きさを平均 0、分散 1 に変換することである。

なお、標準偏差を二乗すると分散となることから、
分散が1であれば、標準偏差も1になっている。

In [ ]:
fig, (ax1, ax2) = plt.subplots( 1, 2 , figsize=(10,4))
sns.histplot( df['年齢'], bins=10, kde=False, ax=ax1)
sns.histplot( df['利用回数'], bins=10, kde=False, ax=ax2)
plt.show()

In [ ]:
df['年齢'].var()

In [ ]:
df['利用回数'].var()

このように、項目によって数値の大きさが異なる場合には標準化が不可欠である。

##【復習】

var()以外にもデータ分析で使う主な関数には以下のものがあった。

関数  | 意味
--    |   --
sum()    | 合計
mean()   | 平均
**var()**    | **分散**
std()    | 標準偏差
median() | 中央値
min()    | 最小値
max()    | 最大値
quantile(0.25)  | 四分位点

## Pythonでの標準化の方法

Pythonでは、データを変換するためのモジュールが用意されている。
データの標準化には、
StandardScaler() を用いる。

使い方は、

1. 準備
2. 訓練（fit） ：変換に必要な平均と分散の計算
3. 変換（transform）：実際に変換を実施する

の3ステップである。


In [ ]:
# 標準化に必要なライブラリーのインポート
from sklearn.preprocessing import StandardScaler

In [ ]:
# 標準化の対象となる項目を変数 X にセットする
X = df[['年齢', '利用回数']]

# 準備
scaler = StandardScaler()

# 訓練の実施
scaler.fit(X)

# 標準化の実施
# 標準化後のデータを再びXにセットする
X = scaler.transform(X)

変換に用いたパラメータ（平均や分散）を確認することもできる。


*   平均：scaler.mean_
*   リスト項目：caler.var_



In [ ]:
scaler.mean_

In [ ]:
scaler.var_

変数 X には2列のデータが格納されていたので、
列ごとに平均と分散が表示される。

## 可視化して確認する

In [ ]:
# 標準化の確認
X = pd.DataFrame( X, columns=['x1', 'x2'])

fig, (ax1, ax2) = plt.subplots( 1, 2 , figsize=(10,4))

ax1.set_title("変換前")
sns.scatterplot(data=df, x='年齢', y='利用回数', ax=ax1 )
ax1.grid(True)  # グリッド線を追加

ax2.set_title("変換後")
sns.scatterplot(data=X, x='x1', y='x2', ax=ax2)
ax2.grid(True)  # グリッド線を追加

plt.show()

##数値で確認する

変換後の平均と分散(標準偏差)を確認する。


In [ ]:
X.describe()

# クラスタリングの実行

## K-means 法

本講義では、クラスタリングを行う代表的な手法であるK-means法を扱う。

データ分析ライブラリーの sklearn に含まれている **KMeans** を用いる。
使い方は、


1.   準備
2.   計算（fit）

の2ステップである。

準備では、いくつかのパラメータを設定する。主なものは、
*   n_clusters
*   init
*   max_iter
*   n_init
である。中でも、もっとも重要なパラメータは

n_clusters

で、クラスタ数を指定指定する必要がある。



詳しい使い方は以下のリンクを参照

https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html


In [ ]:
# クラスタリングに必要なライブラリーのインポート
from sklearn.cluster import KMeans

# 標準化に必要なライブラリーのインポート
from sklearn.preprocessing import StandardScaler

# クラスタリング分析に用いる項目を選び
# 選んだ項目を変数 X にセットする
X = df[ ['年齢', '利用回数']]

# 準備の準備
scaler = StandardScaler()

# 標準化後のデータを再びXにセットする
X = scaler.fit_transform(X)

# 最適なクラスタ数を3であると仮定
best_n_clusters = 3

# 準備
kmeans = KMeans(n_clusters=best_n_clusters, init='k-means++', max_iter=500, n_init='auto')

# クラスタの計算
kmeans.fit(X)

In [ ]:
print(kmeans)

#クラスタ結果の可視化

クラスタリングの結果は、kmeans.labels_ に格納されている。
結果を、元データのデータフレームdfに追加することで、
可視化が容易になる。

In [ ]:
# クラスタラベルをデータフレームに追加
df['cluster'] = kmeans.labels_

In [ ]:
df.head()

In [ ]:
# 結果の可視化（seabornを使用）
sns.scatterplot(x='年齢', y='利用回数', hue='cluster', data=df, palette='Set1')
plt.show()

## クラスター平均

クラスター毎にデータを平均した点をクラスター平均
（クラスター重心）と呼ぶ。

In [ ]:
# クラスタで分類して年齢と利用回数の平均を求める
# クラスタ中心と呼ぶ
df.groupby('cluster')[['年齢','利用回数']].mean()

In [ ]:
c = df.groupby('cluster')[['年齢','利用回数']].mean().reset_index()
sns.scatterplot(x='年齢', y='利用回数', hue='cluster', data=df, palette='Set1')
sns.scatterplot(x='年齢', y='利用回数', data=c, color='k', s=100)

## クラスタ内距離

データがそれぞれにクラスターにどの程度集中しているかを示す指標が
クラスター内距離である。これは、
クラスタ毎にクラスタ中心から各データへの距離の二乗の和である。

In [ ]:
# クラスタ毎にクラスタ中心から各データへの距離の二乗の和
kmeans.inertia_

# クラスタ数の決定

クラスタ数を決定する際には、クラスタ内距離に注目する。

一般に、
クラスタ数を増やすとクラスタ内距離は減少するが、
同程度のクラスタ内距離であれば、
クラスタ数が少なくシンプルなクラスタリングが良いとされる。

In [ ]:
# クラスタ数を 2 にして実行する
best_n_clusters = 2

kmeans = KMeans(n_clusters=best_n_clusters, init='k-means++', max_iter=500, n_init='auto')
kmeans.fit(X)
df['cluster'] = kmeans.labels_

# 図示する
c = df.groupby('cluster')[['年齢','利用回数']].mean().reset_index()
sns.scatterplot(x='年齢', y='利用回数', hue='cluster', data=df, palette='Set1')
sns.scatterplot(x='年齢', y='利用回数', data=c, color='k', s=100)

In [ ]:
# クラスタ毎にクラスタ中心から各データへの距離の二乗の和
kmeans.inertia_

クラスタ数が2の場合では、クラスタ内距離が約35であった。
それと比べると倍以上大きくなってしまうことから、
クラスタ数を2とすることは適切でないと考えられる。

## エルボー法

クラスタ数を決める方法として、エルボー法が使われる。

クラスタ数を変化させながら、クラスタ中心からの距離の二乗の和を計算しグラフ化する。
グラフの減少度合いが変化し始めるところを、最適なクラスター数として採用する。

こういった作業はAIの開発では一般的で、**「ハイパーパラメータチューニング」**と呼ばれる。

In [ ]:
# Elbow Methodによるハイパーパラメータチューニング
inertia = []
for i in range(1, 11):
    kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init='auto')
    kmeans.fit(X)
    inertia.append(kmeans.inertia_)

In [ ]:
# グリッド線を表示するスタイルに変更する
sns.set_style('whitegrid')

# エルボー図のプロット
elbow = pd.DataFrame( inertia, columns = ['Inertia'])
elbow['Num of clusters'] = range(1,11)
sns.lineplot(data = elbow, x='Num of clusters', y= 'Inertia')



---



---



# Wineデータの基礎集計

In [ ]:
from sklearn.datasets import load_wine
# データの読み込み
wine = load_wine()
df = pd.DataFrame(wine.data, columns=wine.feature_names)

In [ ]:
df.info()

## データ概要

Wineデータセットは、イタリアの同じ地域で栽培された3種類のワインに
含まれる成分を測定したデータ。
13の項目がある。

| 項目                          | 説明                                       |
|---------------------------------|--------------------------------------------|
| Alcohol（アルコール）           | ワインに含まれるアルコールの量。           |
| Malic Acid（リンゴ酸）          | ワインに含まれるリンゴ酸の量。             |
| Ash（灰分）                     | ワインの灰分の量。                         |
| Alcalinity of Ash（灰分のアルカリ度） | 灰分のアルカリ度。                   |
| Magnesium（マグネシウム）       | ワインに含まれるマグネシウムの量。         |
| Total Phenols（総フェノール）   | ワインに含まれるフェノール類の総量。       |
| Flavanoids（フラバノイド）      | ワインに含まれるフラバノイドの量。         |
| Nonflavanoid Phenols（非フラバノイドフェノール） | ワインに含まれる非フラバノイドフェノールの量。 |
| Proanthocyanins（プロアントシアニン） | ワインに含まれるプロアントシアニンの量。 |
| Color Intensity（色の強度）     | ワインの色の強度。                         |
| Hue（色相）                     | ワインの色相。                             |
| OD280/OD315 of Diluted Wines（希釈ワインのOD280/OD315） | 希釈されたワインのOD280/OD315の比率。 |
| Proline（プロリン）             | ワインに含まれるプロリンの量。             |

In [ ]:
# 相関行列を計算
correlation_matrix = df.corr().abs()
# sns.heatmap(correlation_matrix, cmap= sns.color_palette('coolwarm', 10), annot=True,fmt='.2f', vmin = -1, vmax = 1)

# 相関が0.5以上の変数を取得
high_corr_var = {}
for i in range(len(correlation_matrix.columns)):
    for j in range(i):
        if abs(correlation_matrix.iloc[i, j]) >= 0.5:
            colname = correlation_matrix.columns[i]
            high_corr_var[colname] = abs(correlation_matrix.iloc[i, j])

# 相関性の高い変数を削除
df.drop(columns=high_corr_var.keys(), inplace=True)

# 結果の表示
print("Remaining columns after removing highly correlated variables:")
print(df.columns)
df.info()

簡単にするため以下の項目限定して分析する。

| 項目                          | 説明                                       |
|---------------------------------|--------------------------------------------|
| Alcohol（アルコール）           | ワインに含まれるアルコールの量。           |
| Malic Acid（リンゴ酸）          | ワインに含まれるリンゴ酸の量。             |
| Ash（灰分）                     | ワインの灰分の量。                         |
| Alcalinity of Ash（灰分のアルカリ度） | 灰分のアルカリ度。                   |
| Magnesium（マグネシウム）       | ワインに含まれるマグネシウムの量。         |
| Total Phenols（総フェノール）   | ワインに含まれるフェノール類の総量。       |


In [ ]:
df = df[['alcohol', 'malic_acid', 'ash', 'alcalinity_of_ash', 'magnesium','total_phenols']]
df.info()

In [ ]:
sns.pairplot( data = df)